In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
from collections import Counter
import liwc
import math
import json
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import arabic_reviews

# Books

In [3]:
books_data = pd.read_csv("Book_labeled_all_rows_with_ratings.csv")
books_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,helpfulness_gold,no,review,rating,ratings_combined
0,2905523768,False,finalized,3,12/23/2020 07:25:58,somewhat_helpful,0.6598,NaN,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,5.0,pos_rating
1,2905523769,False,finalized,3,12/23/2020 07:25:58,helpful,0.6598,NaN,52854,.. .. هنا التقيت بأحد أعجب الرجال اطلاقا،شمس ا...,5.0,pos_rating
2,2905523770,False,finalized,3,12/23/2020 07:25:58,unrelated,0.6598,NaN,48057,سأعود,3.0,neutral_ratings
3,2905523771,False,finalized,3,12/23/2020 07:25:58,helpful,1.0000,NaN,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,2.0,neg_rating
4,2905523772,False,finalized,3,12/23/2020 07:54:33,helpful,1.0000,NaN,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,4.0,pos_rating


In [4]:
books_conf = books_data[(books_data["confidence"] == 1.0) & (books_data["helpfulness"] == "helpful") | (books_data["helpfulness"] != "helpful")]
books_conf.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,helpfulness_gold,no,review,rating,ratings_combined
0,2905523768,False,finalized,3,12/23/2020 07:25:58,somewhat_helpful,0.6598,NaN,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,5.0,pos_rating
2,2905523770,False,finalized,3,12/23/2020 07:25:58,unrelated,0.6598,NaN,48057,سأعود,3.0,neutral_ratings
3,2905523771,False,finalized,3,12/23/2020 07:25:58,helpful,1.0000,NaN,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,2.0,neg_rating
4,2905523772,False,finalized,3,12/23/2020 07:54:33,helpful,1.0000,NaN,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,4.0,pos_rating
5,2905523773,False,finalized,3,12/23/2020 07:54:33,somewhat_helpful,0.6598,NaN,59497,مطولة بشكل مبالغ، لكن تلقي الضوء على جوانب غير...,3.0,neutral_ratings


In [5]:
classes = books_conf["ratings_combined"].map(lambda x: "pos_rating" if x == "pos_rating" else "neg_rating")

In [6]:
books_data = pd.read_csv("appen_translated - books.csv")
books_conf = books_data[(books_data["confidence"] == 1.0) & (books_data["helpfulness"] == "helpful") | (books_data["helpfulness"] != "helpful")]
books_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,review_en
0,2905523768,False,finalized,3,12/23/2020 7:25:58,somewhat_helpful,0.6598,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,Whatever the intellectual tastes .. necessary ...
1,2905523769,False,finalized,3,12/23/2020 7:25:58,helpful,0.6598,52854,.. .. هنا التقيت بأحد أعجب الرجال اطلاقا،شمس ا...,.. .. here I met one impressed by the men at a...
2,2905523770,False,finalized,3,12/23/2020 7:25:58,unrelated,0.6598,48057,سأعود,I'll come back
3,2905523771,False,finalized,3,12/23/2020 7:25:58,helpful,1.0000,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,I had not read the dreams of the famous Thelat...
4,2905523772,False,finalized,3,12/23/2020 7:54:33,helpful,1.0000,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,Language fascinates me and her pen. I wished t...


In [7]:
tf_idf_conf = arabic_reviews.extract_tfidf(books_conf["review_en"])

[nltk_data] Downloading package stopwords to /Users/Omar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Omar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Sample feature names identified :  ['-_-', '-aa', '-aasy-', '-adman', '-adtham', '-aharov', '-ahiana', '-algrayb', '-alhab', '-alkon', '-alm', '-alnhih', '-aloutn', '-alshl', '-alta', '-aly', '-amrh', '-and', '-aqsd', '-are', '-atagaflt', '-azn', '-bbdnha', '-btal', '-d']

Size of TFIDF matrix :  (1480, 13003)


In [14]:
tf_idf_conf.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [32]:
arabic_reviews.ex_with_best_k(tf_idf_conf[0],classes,100)

k = ----------  1
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
SVM
Confusion Matrix : 
[[  1  45]
 [  1 101]]
Acc:  0.689  Prec:  0.692  Recall:  0.99  F1: 0.815
Decision Tree
Confusion Matrix : 
[[  1  45]
 [  2 100]]
Acc:  0.682  Prec:  0.69  Recall:  0.98  F1: 0.81
Random Forest
Confusion Matrix : 
[[  1  45]
 [  2 100]]
Acc:  0.682  Prec:  0.69  Recall:  0.98  F1: 0.81
NN
Confusion Matrix : 
[[  1  45]
 [  2 100]]
Acc:  0.682  Prec:  0.69  Recall:  0.98  F1: 0.81
k = ----------  2
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
SVM
Confusion Matrix : 
[[  1  45]
 [  1 101]]
Acc:  0.689  Prec:  0.692  Recall:  0.99  F1: 0.815
Decision Tree
Confusion Matrix : 
[[ 6 40]
 [ 5 97]]
Acc:  0.696  Prec

Confusion Matrix : 
[[  1  45]
 [  1 101]]
Acc:  0.689  Prec:  0.692  Recall:  0.99  F1: 0.815
NN
Confusion Matrix : 
[[10 36]
 [10 92]]
Acc:  0.689  Prec:  0.719  Recall:  0.902  F1: 0.8
k = ----------  15
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  1  45]
 [  0 102]]
Acc:  0.696  Prec:  0.694  Recall:  1.0  F1: 0.819
SVM
Confusion Matrix : 
[[  2  44]
 [  1 101]]
Acc:  0.696  Prec:  0.697  Recall:  0.99  F1: 0.818
Decision Tree
Confusion Matrix : 
[[11 35]
 [11 91]]
Acc:  0.689  Prec:  0.722  Recall:  0.892  F1: 0.798
Random Forest
Confusion Matrix : 
[[  2  44]
 [  1 101]]
Acc:  0.696  Prec:  0.697  Recall:  0.99  F1: 0.818
NN
Confusion Matrix : 
[[11 35]
 [12 90]]
Acc:  0.682  Prec:  0.72  Recall:  0.882  F1: 0.793
k = ----------  16
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  1  45]
 [  0 102]]
Acc:  0.696  Prec:  0.694  Reca

Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[18 28]
 [10 92]]
Acc:  0.743  Prec:  0.767  Recall:  0.902  F1: 0.829
k = ----------  29
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  1  45]
 [  0 102]]
Acc:  0.696  Prec:  0.694  Recall:  1.0  F1: 0.819
SVM
Confusion Matrix : 
[[  3  43]
 [  1 101]]
Acc:  0.703  Prec:  0.701  Recall:  0.99  F1: 0.821
Decision Tree
Confusion Matrix : 
[[15 31]
 [12 90]]
Acc:  0.709  Prec:  0.744  Recall:  0.882  F1: 0.807
Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[17 29]
 [12 90]]
Acc:  0.723  Prec:  0.756  Recall:  0.882  F1: 0.814
k = ----------  30
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Rec

Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[20 26]
 [13 89]]
Acc:  0.736  Prec:  0.774  Recall:  0.873  F1: 0.82
k = ----------  43
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Recall:  1.0  F1: 0.823
SVM
Confusion Matrix : 
[[  3  43]
 [  2 100]]
Acc:  0.696  Prec:  0.699  Recall:  0.98  F1: 0.816
Decision Tree
Confusion Matrix : 
[[16 30]
 [17 85]]
Acc:  0.682  Prec:  0.739  Recall:  0.833  F1: 0.783
Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[15 31]
 [10 92]]
Acc:  0.723  Prec:  0.748  Recall:  0.902  F1: 0.818
k = ----------  44
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Reca

Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
k = ----------  57
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Recall:  1.0  F1: 0.823
SVM
Confusion Matrix : 
[[  3  43]
 [  2 100]]
Acc:  0.696  Prec:  0.699  Recall:  0.98  F1: 0.816
Decision Tree
Confusion Matrix : 
[[18 28]
 [21 81]]
Acc:  0.669  Prec:  0.743  Recall:  0.794  F1: 0.768
Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[23 23]
 [11 91]]
Acc:  0.77  Prec:  0.798  Recall:  0.892  F1: 0.843
k = ----------  58
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Re

Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[22 24]
 [11 91]]
Acc:  0.764  Prec:  0.791  Recall:  0.892  F1: 0.839
k = ----------  71
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Recall:  1.0  F1: 0.823
SVM
Confusion Matrix : 
[[  3  43]
 [  2 100]]
Acc:  0.696  Prec:  0.699  Recall:  0.98  F1: 0.816
Decision Tree
Confusion Matrix : 
[[19 27]
 [17 85]]
Acc:  0.703  Prec:  0.759  Recall:  0.833  F1: 0.794
Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[22 24]
 [12 90]]
Acc:  0.757  Prec:  0.789  Recall:  0.882  F1: 0.833
k = ----------  72
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Pre

Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[25 21]
 [17 85]]
Acc:  0.743  Prec:  0.802  Recall:  0.833  F1: 0.817
k = ----------  85
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Recall:  1.0  F1: 0.823
SVM
Confusion Matrix : 
[[  3  43]
 [  2 100]]
Acc:  0.696  Prec:  0.699  Recall:  0.98  F1: 0.816
Decision Tree
Confusion Matrix : 
[[22 24]
 [21 81]]
Acc:  0.696  Prec:  0.771  Recall:  0.794  F1: 0.783
Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[24 22]
 [13 89]]
Acc:  0.764  Prec:  0.802  Recall:  0.873  F1: 0.836
k = ----------  86
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Pre

Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[24 22]
 [12 90]]
Acc:  0.77  Prec:  0.804  Recall:  0.882  F1: 0.841
k = ----------  99
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 0 ... 1 1 1]
NB
Confusion Matrix : 
[[  2  44]
 [  0 102]]
Acc:  0.703  Prec:  0.699  Recall:  1.0  F1: 0.823
SVM
Confusion Matrix : 
[[  3  43]
 [  2 100]]
Acc:  0.696  Prec:  0.699  Recall:  0.98  F1: 0.816
Decision Tree
Confusion Matrix : 
[[21 25]
 [21 81]]
Acc:  0.689  Prec:  0.764  Recall:  0.794  F1: 0.779
Random Forest
Confusion Matrix : 
[[  0  46]
 [  0 102]]
Acc:  0.689  Prec:  0.689  Recall:  1.0  F1: 0.816
NN
Confusion Matrix : 
[[22 24]
 [15 87]]
Acc:  0.736  Prec:  0.784  Recall:  0.853  F1: 0.817
0.8108108108108109


In [33]:
classes_3 = books_conf["ratings_combined"]

In [37]:
arabic_reviews.ex_with_best_k(tf_idf_conf[0],classes_3,100)

k = ----------  1
Classes found :  ['neg_rating' 'neutral_ratings' 'pos_rating']

Classes converted to integers : [2 1 0 ... 2 2 2]
NB
Confusion Matrix : 
[[  0   0  18]
 [  0   0  28]
 [  0   0 102]]


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [14]:
top_words(tf_idf_conf[1],tf_idf_conf[0],classes,83,"star_ratings_arabic_books_top_words.csv")

[('boring', 10.17480271616914),
 ('idea', 5.292176738429533),
 ('wonderful', 4.982149952462894),
 ('star', 4.195703367656488),
 ('psychological', 4.030825932687479),
 ('stupid', 3.916928500064742),
 ('naive', 3.8683565574324854),
 ('article', 3.6230512648773194),
 ('-_-', 3.4943462686077353),
 ('weak', 3.419401288923398),
 ('e', 3.2914088954784257),
 ('drug', 3.224299086028083),
 ('painful', 3.193981156033584),
 ('good', 3.1232581243322057),
 ('liked', 3.0165692787138676),
 ('fantastic', 2.8288697829933103),
 ('disappointing', 2.8284356980394945),
 ('bad', 2.805335459487599),
 ('mahbthash', 2.7506289908223933),
 ('lack', 2.6433174380695146),
 ('duplicate', 2.631102400976438),
 ('somewhat', 2.6275973010312605),
 ('silly', 2.579698597949795),
 ('tedious', 2.5132482676200287),
 ('uproar', 2.4881732036638504),
 ('abdulaziz', 2.4799145726230902),
 ('general', 2.4558673574539207),
 ('heart', 2.4178383760996693),
 ('pornographic', 2.4135548556317765),
 ('palestine', 2.291276549389516),
 ('ash